In [14]:
import pandas as pd
import random
import numpy as np
import os

# ==============================================================================
# CONFIGURACIÓN DEL PROYECTO
# ==============================================================================

# Lista de archivos CSV (asume que están en la misma carpeta)
ARCHIVOS_CSV = {
    'QB': '../data/QB.csv',
    'RB': '../data/RB.csv',
    'WR': '../data/WR.csv',
    'TE': '../data/TE.csv',
    'K': '../data/K.csv',
    'DST': '../data/DST.csv'
}

# Configuraciones del Draft y el Modelo
NUM_EQUIPOS = 12
NUM_RONDAS = 14
# Aleatoriedad: % de los mejores jugadores a considerar en cada pick (10% de top talent)
RANDOMNESS_PERCENT = 0.10 
# Columna de Puntos: Usamos 'FPTS' como la columna de proyección de valor del jugador
PUNTOS_COL = 'FPTS'

In [15]:
# ==============================================================================
# 1. CARGA, LIMPIEZA Y CONSOLIDACIÓN DE DATOS DESDE CSV
# ==============================================================================

def cargar_datos_csv(archivos_dict, puntos_col_name):
    """Carga los 6 archivos CSV, los limpia y los consolida en un solo DataFrame."""
    all_dfs = []
    
    print("Iniciando carga de datos desde archivos CSV...")
    
    for pos, filename in archivos_dict.items():
        if not os.path.exists(filename):
            print(f"❌ ERROR: Archivo no encontrado: {filename}. Saltando {pos}.")
            continue
        
        try:
            df = pd.read_csv(filename)
            
            # Limpieza de la columna de jugador (asumiendo formato 'Nombre (Equipo)')
            if 'Player' in df.columns and pos != 'DST':
                df['Player'] = df['Player'].astype(str).str.split('(').str[0].str.strip()
            
            # Renombrar la columna de puntos
            if puntos_col_name in df.columns:
                df = df.rename(columns={puntos_col_name: 'FPTS_PROYECTADOS'})
            elif 'FPTS/G' in df.columns: # Intenta con FPTS/G si la principal no existe
                 df = df.rename(columns={'FPTS/G': 'FPTS_PROYECTADOS'})
            else:
                print(f"❌ ERROR: Columna de puntos ('{puntos_col_name}' o 'FPTS/G') no encontrada en {filename}.")
                continue

            df['Position'] = pos
            all_dfs.append(df)
            
        except Exception as e:
            print(f"❌ ERROR al procesar {filename}: {e}")

    if not all_dfs:
        return pd.DataFrame()

    master_df = pd.concat(all_dfs, ignore_index=True)
    
    # Limpieza final
    master_df['FPTS_PROYECTADOS'] = pd.to_numeric(master_df['FPTS_PROYECTADOS'], errors='coerce')
    master_df = master_df.dropna(subset=['Player', 'Position', 'FPTS_PROYECTADOS'])
    master_df = master_df[master_df['FPTS_PROYECTADOS'] > 0]
    
    # Manejar duplicados (si un jugador aparece en varias tablas, toma el de mayor FPTS)
    master_df = master_df.sort_values(by='FPTS_PROYECTADOS', ascending=False).drop_duplicates(subset=['Player'], keep='first')
    
    print(f"✅ Carga de datos completada. Total de jugadores: {len(master_df)}")
    return master_df[['Player', 'Position', 'FPTS_PROYECTADOS']]

In [16]:
# ==============================================================================
# 2. SIMULACIÓN DEL MOCK DRAFT (Snake Draft con Aleatoriedad)
# ==============================================================================

def simular_mock_draft(player_pool_df, num_teams, num_rounds, randomness):
    """
    Simula un draft snake con aleatoriedad y lógica para forzar posiciones 
    clave (QB, TE, K, DST) si están faltantes en rondas avanzadas.
    """
    draft_pool = player_pool_df.sort_values(by='FPTS_PROYECTADOS', ascending=False).copy()
    draft_pool['Original_Index'] = draft_pool.index
    rosters = {i: [] for i in range(1, num_teams + 1)}
    required_starters = ['QB', 'TE', 'K', 'DST']
    
    print("\n" + "="*80)
    print(f"--- 📝 SIMULACIÓN DE MOCK DRAFT ({num_teams} EQUIPOS, {num_rounds} RONDAS) ---")
    print("="*80)
    
    for ronda in range(1, num_rounds + 1):
        
        # Orden Snake Draft
        draft_order = list(range(1, num_teams + 1)) if ronda % 2 != 0 else list(range(num_teams, 0, -1))
        # print(f"Ronda {ronda:02d} | Orden: {draft_order[0]} -> {draft_order[-1]}")

        for team_id in draft_order:
            if draft_pool.empty or len(rosters[team_id]) >= num_rounds:
                continue

            current_roster_positions = set([p['Position'] for p in rosters[team_id]])
            
            # --- Lógica de Selección ---
            forced_pick = None
            if ronda > 4 and len(rosters[team_id]) < num_rounds: 
                for pos in required_starters:
                    if pos not in current_roster_positions:
                        pos_pool = draft_pool[draft_pool['Position'] == pos]
                        if not pos_pool.empty:
                            forced_pick = pos_pool.head(1).iloc[0]
                            break
            
            if forced_pick is not None:
                pick = forced_pick
                pick_index = pick['Original_Index']
            else:
                # Selección con Aleatoriedad (Top N% de jugadores)
                n_picks_consider = max(1, int(len(draft_pool) * randomness))
                available_picks = draft_pool.head(n_picks_consider)
                
                pick_index_df = random.choice(available_picks.index.tolist())
                pick = draft_pool.loc[pick_index_df]
                pick_index = pick['Original_Index']
            
            # 3. Registrar el Pick
            rosters[team_id].append(pick.drop('Original_Index').to_dict())
            
            # 4. Eliminar el jugador seleccionado del pool de draft
            draft_pool = draft_pool[draft_pool['Original_Index'] != pick_index].reset_index(drop=True)
    
    final_rosters = {team_id: pd.DataFrame(picks).reset_index(drop=True) for team_id, picks in rosters.items()}
    return final_rosters

In [17]:
# ==============================================================================
# 3. ALGORITMO GREEDY DE OPTIMIZACIÓN DEL LINEUP
# ==============================================================================

def optimizar_lineup_greedy(roster_df):
    """
    Algoritmo Greedy que selecciona el mejor jugador para cada puesto 
    fijo y luego el mejor jugador restante para el FLEX.
    """
    roster_copy = roster_df.copy()
    roster_copy['Original_Index'] = roster_copy.index 
    roster_copy = roster_copy.sort_values(by='FPTS_PROYECTADOS', ascending=False)
    
    starters = []
    selected_indices = set()
    
    fixed_positions = {'QB': 1, 'K': 1, 'DST': 1}
    variable_positions_slots = {'RB': 2, 'WR': 2, 'TE': 1}
    
    # 1. Llenar Posiciones Fijas
    for pos, count in fixed_positions.items():
        best_player = roster_copy[roster_copy['Position'] == pos]
        best_player = best_player[~best_player['Original_Index'].isin(selected_indices)].head(count)
        
        if not best_player.empty:
            starters.extend(best_player.to_dict('records'))
            selected_indices.update(best_player['Original_Index'].tolist())

    # 2. Llenar Posiciones Variables Mínimas
    for pos, count in variable_positions_slots.items():
        best_players = roster_copy[roster_copy['Position'] == pos]
        best_players = best_players[~best_players['Original_Index'].isin(selected_indices)].head(count)
        
        if not best_players.empty:
            starters.extend(best_players.to_dict('records'))
            selected_indices.update(best_players['Original_Index'].tolist())
            
    # 3. Llenar el puesto FLEX (El mejor jugador restante de RB, WR o TE)
    flex_pool = roster_copy[roster_copy['Position'].isin(['RB', 'WR', 'TE'])]
    flex_player = flex_pool[~flex_pool['Original_Index'].isin(selected_indices)].head(1)
    
    if not flex_player.empty:
        starters.extend(flex_player.to_dict('records'))
        selected_indices.update(flex_player['Original_Index'].tolist())

    # 4. Determinar la Banca (bench) y Puntuación Total
    starters_df = pd.DataFrame(starters)
    bench_df = roster_df.loc[~roster_df.index.isin(selected_indices)].reset_index(drop=True)
    total_fpts = starters_df['FPTS_PROYECTADOS'].sum()
    
    return starters_df, bench_df, total_fpts

In [18]:
# ==============================================================================
# 4. FUNCIÓN PRINCIPAL DE EJECUCIÓN
# ==============================================================================

def main():
    
    # 1. Carga de Datos
    player_pool_df = cargar_datos_csv(ARCHIVOS_CSV, PUNTOS_COL)
    
    if player_pool_df.empty:
        print("\nEl proceso ha finalizado debido a errores en la carga o limpieza de datos.")
        return

    # 2. Simulación del Mock Draft
    final_rosters = simular_mock_draft(player_pool_df, NUM_EQUIPOS, NUM_RONDAS, RANDOMNESS_PERCENT)
    
    # 3. Elegir el Equipo 1 para la optimización
    team_id_to_optimize = 1
    roster_to_optimize = final_rosters[team_id_to_optimize]
    
    # 4. Optimizar el Lineup del equipo seleccionado
    starters_df, bench_df, total_fpts = optimizar_lineup_greedy(roster_to_optimize)
    
    # 5. Generar y Mostrar Resultados
    
    roster_to_optimize_output = roster_to_optimize[['Player', 'Position', 'FPTS_PROYECTADOS']].sort_values(by='FPTS_PROYECTADOS', ascending=False)
    
    print("\n" + "="*80)
    print(f"ROSTER FINAL DEL EQUIPO {team_id_to_optimize} (14 jugadores):")
    print("="*80)
    print(roster_to_optimize_output.to_string(index=False))
    
    print("\n" + "="*80)
    
    if len(starters_df) < 9:
        print(f"❌ ERROR: El lineup no pudo ser completado (solo {len(starters_df)} titulares).")
        return
        
    print(f"✅ LINEUP ÓPTIMO (Greedy) GENERADO | Puntuación Total Proyectada: {total_fpts:.2f} FPTS")
    print("="*80)
    
    # Asignación de Posiciones para la presentación final
    starters_df = starters_df.sort_values(by='FPTS_PROYECTADOS', ascending=False)
    output_lines = []
    assigned_indices = set()
    
    # Función de ayuda para la presentación
    def get_player_info(df, pos, rank):
        filtered = df[df['Position'] == pos].iloc[rank-1:rank]
        if not filtered.empty:
            player = filtered.iloc[0]
            return player['Player'], player['FPTS_PROYECTADOS'], filtered.index[0]
        return None, None, None

    # QB
    player, fpts, idx = get_player_info(starters_df, 'QB', 1)
    output_lines.append(f"| QB:  {player} ({fpts:.2f})" if player else f"| QB:  [NO DISPONIBLE]"); 
    if idx is not None: assigned_indices.add(idx)

    # RB1, RB2
    for i in range(1, 3):
        player, fpts, idx = get_player_info(starters_df, 'RB', i)
        output_lines.append(f"| RB{i}: {player} ({fpts:.2f})" if player else f"| RB{i}: [NO DISPONIBLE]");
        if idx is not None: assigned_indices.add(idx)

    # WR1, WR2
    for i in range(1, 3):
        player, fpts, idx = get_player_info(starters_df, 'WR', i)
        output_lines.append(f"| WR{i}: {player} ({fpts:.2f})" if player else f"| WR{i}: [NO DISPONIBLE]");
        if idx is not None: assigned_indices.add(idx)
    
    # TE, K, DST
    for pos, name in [('TE', 'TE'), ('K', 'K'), ('DST', 'DST')]:
        player, fpts, idx = get_player_info(starters_df, pos, 1)
        output_lines.append(f"| {name}:  {player} ({fpts:.2f})" if player else f"| {name}:  [NO DISPONIBLE]");
        if idx is not None: assigned_indices.add(idx)

    # Identificar el jugador FLEX
    flex_player_df = starters_df.loc[~starters_df.index.isin(assigned_indices)].head(1)
    
    if not flex_player_df.empty:
         flex_player = flex_player_df.iloc[0]
         output_lines.append(f"| FLEX: {flex_player['Player']} ({flex_player['Position']}) ({flex_player['FPTS_PROYECTADOS']:.2f})")
    
    print("\n--- 🏈 STARTING LINEUP ÓPTIMO (10 Jugadores) ---")
    print('\n'.join(output_lines))
    
    # Mostrar la Banca
    print("\n--- 🪑 JUGADORES EN LA BANCA (4 Jugadores) ---")
    bench_output = bench_df[['Player', 'Position', 'FPTS_PROYECTADOS']].sort_values(by='FPTS_PROYECTADOS', ascending=False)
    print(bench_output.to_string(index=False))

if __name__ == "__main__":
    main()

Iniciando carga de datos desde archivos CSV...
✅ Carga de datos completada. Total de jugadores: 456

--- 📝 SIMULACIÓN DE MOCK DRAFT (12 EQUIPOS, 14 RONDAS) ---

ROSTER FINAL DEL EQUIPO 1 (14 jugadores):
                   Player Position  FPTS_PROYECTADOS
           Caleb Williams       QB              84.1
             Dak Prescott       QB              75.9
         Javonte Williams       RB              60.6
           George Pickens       WR              54.0
             J.K. Dobbins       RB              52.1
            Chuba Hubbard       RB              43.1
              Romeo Doubs       WR              41.9
             Cam Skattebo       RB              41.9
               Cam Little        K              39.0
             Hunter Henry       TE              38.4
Pittsburgh Steelers (PIT)      DST              33.0
           Daniel Carlson        K              33.0
         Jameson Williams       WR              27.4
           Rashid Shaheed       WR              23.9

✅